对y=2x^Tx 关于列向量 x求导 

In [28]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算y关于向量x之前，需要一个地方保存梯度

In [29]:
x.requires_grad_(True)  # 设为True，表示，要将x的梯度保存起来
# 等价于 x = torch.arange(4.0, require_grad=True)

print(x.grad) # 访问梯度， 默认为None

None


计算y，向量点乘：torch.dot()

In [30]:
y = 2 * torch.dot(x, x)  # Pytorch隐式创建计算图，grad_fn=<MulBackward0>保存梯度函数，这个里面保存了y是x的函数的信息
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数(.backward())，自动计算y关于x每个分量的梯度

In [31]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [32]:
x.grad == 4*x # 4*x就是y对x求导的结果 

tensor([True, True, True, True])

默认情况，Pytorch会累积梯度，所以如果计算新的，需要清除之前的梯度

In [33]:
y = x.sum()
y.backward()
x.grad # 此时x的梯度是错误的，因为y=sum(x)对x求导，梯度应该是全1，现在的结果是因为保存了前面的 0 4 8 12

tensor([ 1.,  5.,  9., 13.])

In [34]:
x.grad.zero_() # 清空现在的梯度
y = x.sum()
y.backward()
x.grad 

tensor([1., 1., 1., 1.])


y如果是向量： 例如y=x*x
按理说向量y对向量x求导，是一个矩阵，但深度学习中我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [35]:
# 对非标量调用`backward()`需要传入一个`gradient` 参数，用于指定微分函数
x.grad.zero_()
y = x*x
# 机器学习，深度学习中很少会对向量去求导，一般都是对标量求backward(loss.backward()) 
# 所以这里做一个求和，变成标量
y.sum().backward()  # 为什么sum()????? 因为sum()的导数是1么?这样又不影响结果，又能变成标量
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外
有啥用？？？？？

In [46]:
x.grad.zero_()
y = x * x
u = y.detach() # detach后，把y当一个常数，而不是关于x的函数
z = u*x

z.sum().backward() # z对x求导，把u当一个常数进行
x.grad == u # 当在网络中需要把一些参数固定住时很有用

tensor([True, True, True, True])

In [47]:
x.grad.zero_()
y.sum().backward()
y, x, x.grad, x.grad ==2*x # y=x^2的导数是2x

(tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 2., 4., 6.]),
 tensor([True, True, True, True]))

y如果不是一个简单的函数，而是一个python控制流(例如 条件。循环或任意函数调用)，也一样可以用backward求导，得到变量的梯度

In [76]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    # 控制流里每个分支都是b的函数，所以可以反向传播求梯度
    # 这也是隐式构造计算图比显示构造的好处，控制流也可以直接求导，如果是显示的话，必须提前定义好变量，那这种分支可能就不方便了
    # 但反过来讲，这样比显示的慢一些
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True) 
# d = f(torch.tensor(1000.))
d = f(a)
d.backward()

# 为什么a的梯度(d对a求导)是d/a?
# 因为：d是a的线性函数，d=ma, 所以d对a的导数为m，而m等于d/a，所以d对a的导数就是d/a

a,d, d/a, a.grad == d/a, a.grad 

(tensor(-0.1055, requires_grad=True),
 tensor(-172922.4375, grad_fn=<MulBackward0>),
 tensor(1638400., grad_fn=<DivBackward0>),
 tensor(True),
 tensor(1638400.))

In [58]:
 # torch.randn(size=()).shape   # size=()生成一个随机标量

In [79]:
def f(a):
    b = a ** 2  # !!!!!如果改为b=a^2!!!!!!!!!!
    while b.norm() < 1000:
        b = b * 2
    # 控制流里每个分支都是b的函数，所以可以反向传播求梯度
    # 这也是隐式构造计算图比显示构造的好处，控制流也可以直接求导，如果是显示的话，必须提前定义好变量，那这种分支可能就不方便了
    # 但反过来讲，这样比显示的慢一些
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True) 
# d = f(torch.tensor(1000.))
d = f(a)
d.backward()

# d=ma^2 d对a的导数为2ma，而2ma等于d/a*2，所以a的梯度为d/a*2
a.grad==d/a*2

tensor(True)